# 服务器使用和工作流

本教程介绍如何在服务器上运行机器学习代码，包括私有集群和使用SLURM的公共集群。

## 内容概览
1. 服务器连接和基本操作
2. 文件传输（上传/下载）
3. 环境配置和包管理
4. 任务提交和监控
5. SLURM作业调度系统
6. 最佳实践和常见问题

## 1. 服务器连接

### SSH连接
```bash
# 基本连接
ssh username@server_address

# 指定端口
ssh -p 2222 username@server_address

# 使用密钥连接
ssh -i ~/.ssh/private_key username@server_address

# 保持连接活跃
ssh -o ServerAliveInterval=60 username@server_address
```

### 配置SSH免密登录
```bash
# 1. 生成SSH密钥对
ssh-keygen -t rsa -b 4096 -C "your_email@example.com"

# 2. 复制公钥到服务器
ssh-copy-id username@server_address

# 3. 或手动添加
cat ~/.ssh/id_rsa.pub | ssh username@server_address "mkdir -p ~/.ssh && cat >> ~/.ssh/authorized_keys"
```

## 2. 文件传输

### SCP命令
```bash
# 上传文件
scp local_file.py username@server:/remote/path/

# 上传文件夹
scp -r local_folder/ username@server:/remote/path/

# 下载文件
scp username@server:/remote/file.py ./local_path/

# 下载文件夹
scp -r username@server:/remote/folder/ ./local_path/
```

### Rsync同步
```bash
# 增量同步（推荐）
rsync -avz --progress local_folder/ username@server:/remote/path/

# 排除特定文件
rsync -avz --exclude='*.log' --exclude='__pycache__' local_folder/ username@server:/remote/path/
```

In [ ]:
# 服务器环境检查脚本
import subprocess
import sys
import os
import psutil

def check_server_environment():
    """检查服务器环境信息"""
    print("=== 服务器环境信息 ===")
    
    # 系统信息
    print(f"操作系统: {os.uname().sysname} {os.uname().release}")
    print(f"主机名: {os.uname().nodename}")
    print(f"Python版本: {sys.version}")
    
    # 硬件信息
    print(f"CPU核心数: {psutil.cpu_count()}")
    print(f"内存总量: {psutil.virtual_memory().total / (1024**3):.1f} GB")
    print(f"可用内存: {psutil.virtual_memory().available / (1024**3):.1f} GB")
    
    # GPU信息
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.used', '--format=csv,noheader,nounits'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            print("GPU信息:")
            for line in result.stdout.strip().split('\n'):
                print(f"  {line}")
        else:
            print("未检测到GPU或nvidia-smi不可用")
    except FileNotFoundError:
        print("nvidia-smi命令未找到")
    
    # 磁盘使用情况
    disk_usage = psutil.disk_usage('/')
    print(f"磁盘使用: {disk_usage.used / (1024**3):.1f} GB / {disk_usage.total / (1024**3):.1f} GB")
    
    # 环境变量
    important_vars = ['CUDA_VISIBLE_DEVICES', 'PATH', 'PYTHONPATH']
    print("\n重要环境变量:")
    for var in important_vars:
        value = os.environ.get(var, '未设置')
        print(f"  {var}: {value[:100]}{'...' if len(str(value)) > 100 else ''}")

# 运行检查
check_server_environment()

## 3. SSH连接管理和配置

### 3.1 SSH配置文件

创建 `~/.ssh/config` 文件来简化连接管理：

In [ ]:
import os
import subprocess
from pathlib import Path
import time
import json

def create_ssh_config():
    """创建SSH配置文件示例"""
    
    ssh_config_content = """
# ~/.ssh/config 示例文件

# 私有集群服务器
Host gpu-server1
    HostName 192.168.1.100
    User your_username
    Port 22
    IdentityFile ~/.ssh/id_rsa
    ServerAliveInterval 60
    ServerAliveCountMax 3
    
Host gpu-server2
    HostName gpu2.university.edu
    User research_account
    Port 2222
    IdentityFile ~/.ssh/gpu_server_key
    ServerAliveInterval 60
    
# 公共集群（SLURM）
Host hpc-cluster
    HostName login.hpc.university.edu
    User student_id
    Port 22
    IdentityFile ~/.ssh/hpc_key
    ServerAliveInterval 120
    
# 跳板机配置
Host internal-server
    HostName 10.0.0.50
    User admin
    ProxyJump gateway-server
    
Host gateway-server
    HostName gateway.company.com
    User gateway_user
    Port 2222

# 通用设置
Host *
    UseKeychain yes
    AddKeysToAgent yes
    Compression yes
    """
    
    print("SSH配置文件示例：")
    print(ssh_config_content)
    
    print("\n🔧 使用方法：")
    print("1. 复制上述内容到 ~/.ssh/config")
    print("2. 修改相应的主机名、用户名和密钥路径")
    print("3. 然后就可以直接使用: ssh gpu-server1")

def test_ssh_connection(host_alias):
    """测试SSH连接"""
    try:
        print(f"🔄 测试连接到 {host_alias}...")
        
        # 测试连接（不执行命令，只测试连通性）
        result = subprocess.run(
            ['ssh', '-o', 'ConnectTimeout=10', '-o', 'BatchMode=yes', 
             host_alias, 'echo "Connection successful"'],
            capture_output=True, text=True, timeout=15
        )
        
        if result.returncode == 0:
            print(f"✅ 成功连接到 {host_alias}")
            return True
        else:
            print(f"❌ 连接失败: {result.stderr}")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"⏱️ 连接超时: {host_alias}")
        return False
    except Exception as e:
        print(f"❌ 连接错误: {e}")
        return False

def generate_ssh_key():
    """生成SSH密钥对"""
    print("🔑 SSH密钥生成指南：")
    
    commands = [
        "# 1. 生成RSA密钥对（推荐4096位）",
        "ssh-keygen -t rsa -b 4096 -C 'your_email@example.com'",
        "",
        "# 2. 生成ED25519密钥对（更安全，推荐）",
        "ssh-keygen -t ed25519 -C 'your_email@example.com'",
        "",
        "# 3. 为特定服务器生成专用密钥",
        "ssh-keygen -t rsa -b 4096 -f ~/.ssh/gpu_server_key -C 'gpu_server_access'",
        "",
        "# 4. 复制公钥到服务器",
        "ssh-copy-id -i ~/.ssh/id_rsa.pub username@server_address",
        "",
        "# 5. 手动复制公钥（如果ssh-copy-id不可用）",
        "cat ~/.ssh/id_rsa.pub | ssh username@server 'mkdir -p ~/.ssh && cat >> ~/.ssh/authorized_keys'"
    ]
    
    for cmd in commands:
        print(cmd)

# 运行示例
create_ssh_config()
print("\n" + "="*60)
generate_ssh_key()

# 注意：实际测试连接需要真实的服务器配置
# test_ssh_connection("gpu-server1")

## 4. 高效文件传输

### 4.1 文件传输工具对比

| 工具 | 特点 | 适用场景 | 命令示例 |
|------|------|----------|----------|
| `scp` | 简单直接 | 小文件，一次性传输 | `scp file.py user@server:/path/` |
| `rsync` | 增量同步，断点续传 | 大文件，频繁同步 | `rsync -avz --progress local/ user@server:/path/` |
| `sftp` | 交互式文件管理 | 文件浏览和管理 | `sftp user@server` |
| `git` | 版本控制 | 代码同步 | `git push/pull` |

### 4.2 文件传输最佳实践

In [ ]:
import shutil
import hashlib
from datetime import datetime

class FileTransferManager:
    """文件传输管理器"""
    
    def __init__(self, server_config):
        self.server_config = server_config
        
    def generate_rsync_command(self, local_path, remote_path, 
                              exclude_patterns=None, dry_run=False):
        """生成rsync命令"""
        if exclude_patterns is None:
            exclude_patterns = [
                '*.pyc', '__pycache__/', '*.log', '.git/', 
                '.vscode/', '.idea/', '*.DS_Store', '*.tmp'
            ]
        
        # 基础命令
        cmd = ['rsync', '-avz', '--progress']
        
        # 添加排除模式
        for pattern in exclude_patterns:
            cmd.extend(['--exclude', pattern])
        
        # 干运行模式
        if dry_run:
            cmd.append('--dry-run')
        
        # 添加路径
        cmd.append(local_path)
        cmd.append(f"{self.server_config['user']}@{self.server_config['host']}:{remote_path}")
        
        return cmd
    
    def upload_project(self, project_path, remote_base_path, dry_run=True):
        """上传项目到服务器"""
        print(f"📤 准备上传项目: {project_path}")
        print(f"🎯 目标路径: {remote_base_path}")
        
        # 生成命令
        cmd = self.generate_rsync_command(
            f"{project_path}/", remote_base_path, dry_run=dry_run
        )
        
        print(f"\n🔧 执行命令:")
        print(" ".join(cmd))
        
        if dry_run:
            print("\n⚠️  这是干运行模式，实际不会传输文件")
            print("   移除 dry_run=True 参数来执行实际传输")
        
        return cmd
    
    def download_results(self, remote_path, local_path, patterns=None):
        """下载结果文件"""
        if patterns is None:
            patterns = ['*.pth', '*.pkl', '*.json', '*.png', '*.jpg', '*.log']
        
        print(f"📥 下载结果文件...")
        
        commands = []
        for pattern in patterns:
            cmd = [
                'rsync', '-avz', '--progress',
                f"{self.server_config['user']}@{self.server_config['host']}:{remote_path}/{pattern}",
                local_path
            ]
            commands.append(cmd)
        
        return commands
    
    def sync_code_only(self, local_path, remote_path, dry_run=True):
        """只同步代码文件"""
        code_patterns = ['*.py', '*.ipynb', '*.yml', '*.yaml', '*.json', '*.txt', '*.md']
        
        cmd = ['rsync', '-avz', '--progress']
        
        # 只包含代码文件
        for pattern in code_patterns:
            cmd.extend(['--include', pattern])
        
        cmd.extend(['--include', '*/'])  # 包含目录
        cmd.append('--exclude=*')  # 排除其他所有文件
        
        if dry_run:
            cmd.append('--dry-run')
            
        cmd.append(f"{local_path}/")
        cmd.append(f"{self.server_config['user']}@{self.server_config['host']}:{remote_path}")
        
        return cmd

def create_transfer_script():
    """创建文件传输脚本"""
    
    script_content = '''#!/bin/bash
# 文件传输脚本 - transfer.sh

# 配置
SERVER_USER="your_username"
SERVER_HOST="gpu-server1"
LOCAL_PROJECT_PATH="./my_ml_project"
REMOTE_BASE_PATH="/home/$SERVER_USER/projects"

# 颜色输出
RED='\\033[0;31m'
GREEN='\\033[0;32m'
YELLOW='\\033[1;33m'
NC='\\033[0m' # No Color

echo_info() { echo -e "${GREEN}[INFO]${NC} $1"; }
echo_warn() { echo -e "${YELLOW}[WARN]${NC} $1"; }
echo_error() { echo -e "${RED}[ERROR]${NC} $1"; }

# 函数：上传项目
upload_project() {
    echo_info "开始上传项目..."
    
    rsync -avz --progress \\
        --exclude='*.pyc' \\
        --exclude='__pycache__/' \\
        --exclude='*.log' \\
        --exclude='.git/' \\
        --exclude='*.DS_Store' \\
        --exclude='data/raw/' \\
        --exclude='checkpoints/' \\
        "$LOCAL_PROJECT_PATH/" \\
        "$SERVER_USER@$SERVER_HOST:$REMOTE_BASE_PATH/$(basename $LOCAL_PROJECT_PATH)/"
    
    if [ $? -eq 0 ]; then
        echo_info "项目上传完成!"
    else
        echo_error "项目上传失败!"
        exit 1
    fi
}

# 函数：下载结果
download_results() {
    echo_info "下载结果文件..."
    
    local remote_project="$REMOTE_BASE_PATH/$(basename $LOCAL_PROJECT_PATH)"
    local local_results="./results/$(date +%Y%m%d_%H%M%S)"
    
    mkdir -p "$local_results"
    
    # 下载模型文件
    rsync -avz --progress \\
        "$SERVER_USER@$SERVER_HOST:$remote_project/checkpoints/" \\
        "$local_results/checkpoints/" 2>/dev/null || echo_warn "没有找到checkpoints目录"
    
    # 下载日志文件
    rsync -avz --progress \\
        "$SERVER_USER@$SERVER_HOST:$remote_project/logs/" \\
        "$local_results/logs/" 2>/dev/null || echo_warn "没有找到logs目录"
    
    # 下载结果图像
    rsync -avz --progress \\
        "$SERVER_USER@$SERVER_HOST:$remote_project/*.png" \\
        "$local_results/" 2>/dev/null || echo_warn "没有找到PNG文件"
        
    echo_info "结果已下载到: $local_results"
}

# 函数：快速同步代码
sync_code() {
    echo_info "快速同步代码文件..."
    
    rsync -avz --progress \\
        --include='*.py' \\
        --include='*.ipynb' \\
        --include='*.yml' \\
        --include='*.yaml' \\
        --include='*.json' \\
        --include='*.txt' \\
        --include='*.md' \\
        --include='*/' \\
        --exclude='*' \\
        "$LOCAL_PROJECT_PATH/" \\
        "$SERVER_USER@$SERVER_HOST:$REMOTE_BASE_PATH/$(basename $LOCAL_PROJECT_PATH)/"
    
    echo_info "代码同步完成!"
}

# 主程序
case "$1" in
    "upload"|"up")
        upload_project
        ;;
    "download"|"down")
        download_results
        ;;
    "sync")
        sync_code
        ;;
    "all")
        upload_project
        echo_info "项目上传完成，现在可以在服务器上运行训练"
        ;;
    *)
        echo "用法: $0 {upload|download|sync|all}"
        echo "  upload/up   - 上传完整项目"
        echo "  download/down - 下载结果文件"
        echo "  sync        - 快速同步代码"
        echo "  all         - 上传项目"
        exit 1
        ;;
esac
'''
    
    print("📝 文件传输脚本 (transfer.sh):")
    print(script_content)
    
    print("\n🔧 使用方法:")
    print("1. 保存上述内容为 transfer.sh")
    print("2. chmod +x transfer.sh")
    print("3. 修改脚本中的服务器配置")
    print("4. ./transfer.sh upload  # 上传项目")
    print("5. ./transfer.sh download  # 下载结果")

# 示例配置
server_config = {
    'user': 'your_username',
    'host': 'gpu-server1',
    'port': 22
}

# 创建传输管理器
transfer_manager = FileTransferManager(server_config)

# 演示各种传输命令
print("📋 文件传输命令示例:\n")

# 1. 上传项目
cmd = transfer_manager.upload_project("./my_ml_project", "/home/user/projects", dry_run=True)
print()

# 2. 下载结果
cmds = transfer_manager.download_results("/home/user/projects/my_ml_project", "./results")
print("📥 下载结果命令:")
for i, cmd in enumerate(cmds, 1):
    print(f"{i}. {' '.join(cmd)}")
print()

# 3. 代码同步
cmd = transfer_manager.sync_code_only("./my_ml_project", "/home/user/projects/my_ml_project")
print("🔄 代码同步命令:")
print(" ".join(cmd))

print("\n" + "="*60)
create_transfer_script()

## 5. 服务器环境配置

### 5.1 虚拟环境管理

在服务器上管理Python环境是关键技能，特别是在共享服务器上：

### 5.2 Conda环境配置

In [ ]:
class ServerEnvironmentManager:
    """服务器环境管理器"""
    
    def __init__(self):
        self.conda_envs = []
        self.system_info = {}
    
    def generate_conda_setup_script(self, env_name="ml_research"):
        """生成Conda环境设置脚本"""
        
        script = f"""
#!/bin/bash
# Conda环境设置脚本 - setup_env.sh

ENV_NAME="{env_name}"

echo "🐍 创建Conda环境: $ENV_NAME"

# 1. 创建新环境
conda create -n $ENV_NAME python=3.9 -y

# 2. 激活环境
source activate $ENV_NAME
# 或者使用: conda activate $ENV_NAME

# 3. 安装基础包
echo "📦 安装基础科学计算包..."
conda install numpy pandas matplotlib seaborn jupyter -y
conda install scikit-learn -y

# 4. 安装深度学习框架
echo "🤖 安装PyTorch..."
# CPU版本
# conda install pytorch torchvision torchaudio cpuonly -c pytorch -y

# GPU版本 (根据CUDA版本选择)
# CUDA 11.8
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y

# CUDA 12.1
# conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia -y

# 5. 安装其他常用包
echo "🔧 安装其他工具包..."
pip install tensorboard
pip install tqdm
pip install wandb  # 实验跟踪
pip install hydra-core  # 配置管理
pip install black flake8  # 代码格式化

# 6. 验证安装
echo "✅ 验证安装..."
python -c "import torch; print(f'PyTorch版本: {{torch.__version__}}'); print(f'CUDA可用: {{torch.cuda.is_available()}}')"

# 7. 保存环境配置
conda env export > environment.yml
echo "💾 环境配置已保存到 environment.yml"

echo "🎉 环境设置完成!"
echo "📝 使用方法:"
echo "   conda activate $ENV_NAME"
"""
        
        return script
    
    def generate_requirements_script(self):
        """生成requirements.txt管理脚本"""
        
        script = """
#!/bin/bash
# Python包管理脚本 - manage_packages.sh

# 函数：导出当前环境
export_env() {
    echo "📋 导出当前环境到 requirements.txt..."
    pip freeze > requirements.txt
    echo "✅ 导出完成: requirements.txt"
    
    echo "📋 导出Conda环境..."
    conda env export > environment.yml
    echo "✅ 导出完成: environment.yml"
}

# 函数：从requirements.txt安装
install_from_requirements() {
    if [ -f "requirements.txt" ]; then
        echo "📦 从 requirements.txt 安装包..."
        pip install -r requirements.txt
        echo "✅ 安装完成!"
    else
        echo "❌ requirements.txt 文件不存在"
    fi
}

# 函数：从environment.yml创建环境
create_from_yml() {
    if [ -f "environment.yml" ]; then
        echo "🐍 从 environment.yml 创建环境..."
        conda env create -f environment.yml
        echo "✅ 环境创建完成!"
    else
        echo "❌ environment.yml 文件不存在"
    fi
}

# 函数：更新环境
update_env() {
    echo "🔄 更新Conda环境..."
    conda update --all -y
    
    echo "🔄 更新pip包..."
    pip list --outdated --format=json | jq -r '.[] | .name' | xargs -I {} pip install --upgrade {}
    
    echo "✅ 环境更新完成!"
}

# 函数：清理环境
clean_env() {
    echo "🧹 清理Conda缓存..."
    conda clean --all -y
    
    echo "🧹 清理pip缓存..."
    pip cache purge
    
    echo "✅ 清理完成!"
}

# 主程序
case "$1" in
    "export")
        export_env
        ;;
    "install")
        install_from_requirements
        ;;
    "create")
        create_from_yml
        ;;
    "update")
        update_env
        ;;
    "clean")
        clean_env
        ;;
    *)
        echo "用法: $0 {export|install|create|update|clean}"
        echo "  export  - 导出当前环境"
        echo "  install - 从requirements.txt安装"
        echo "  create  - 从environment.yml创建环境"
        echo "  update  - 更新所有包"
        echo "  clean   - 清理缓存"
        exit 1
        ;;
esac
"""
        
        return script
    
    def check_gpu_environment(self):
        """检查GPU环境配置"""
        
        check_commands = {
            "NVIDIA驱动": "nvidia-smi",
            "CUDA版本": "nvcc --version",
            "PyTorch GPU": "python -c 'import torch; print(torch.cuda.is_available())'",
            "GPU数量": "python -c 'import torch; print(torch.cuda.device_count())'",
            "GPU名称": "python -c 'import torch; print([torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])'"
        }
        
        print("🔍 GPU环境检查命令:")
        for desc, cmd in check_commands.items():
            print(f"  {desc}: {cmd}")
        
        return check_commands
    
    def generate_bashrc_config(self):
        """生成.bashrc配置"""
        
        bashrc_content = """
# 添加到 ~/.bashrc 的内容

# Conda配置
export PATH="$HOME/miniconda3/bin:$PATH"
# 或者如果使用Anaconda
# export PATH="$HOME/anaconda3/bin:$PATH"

# CUDA配置 (根据实际CUDA安装路径修改)
export CUDA_HOME=/usr/local/cuda
export PATH=$CUDA_HOME/bin:$PATH
export LD_LIBRARY_PATH=$CUDA_HOME/lib64:$LD_LIBRARY_PATH

# Python配置
export PYTHONPATH=$HOME/projects:$PYTHONPATH

# 别名设置
alias ll='ls -alF'
alias la='ls -A'
alias l='ls -CF'
alias grep='grep --color=auto'

# GPU相关别名
alias gpustat='nvidia-smi'
alias gpuwatch='watch -n 1 nvidia-smi'
alias gpufree='nvidia-smi --query-gpu=index,name,memory.free --format=csv'

# Conda相关别名
alias ca='conda activate'
alias cda='conda deactivate'
alias cenv='conda env list'

# 项目相关
alias projects='cd $HOME/projects'
alias logs='cd $HOME/projects && find . -name "*.log" -mtime -1'

# 显示系统信息
echo "🖥️  服务器: $(hostname)"
echo "🐍 Python: $(python --version 2>&1 | head -1)"
if command -v nvidia-smi &> /dev/null; then
    echo "🎮 GPU: $(nvidia-smi --query-gpu=name --format=csv,noheader | head -1)"
fi
echo "📁 当前目录: $(pwd)"

# 自动激活默认环境 (可选)
# conda activate ml_research
"""
        
        return bashrc_content

# 创建环境管理器
env_manager = ServerEnvironmentManager()

print("🐍 Conda环境设置脚本:")
print("="*60)
conda_script = env_manager.generate_conda_setup_script("ml_research")
print(conda_script)

print("\n📦 包管理脚本:")
print("="*60)
packages_script = env_manager.generate_requirements_script()
print(packages_script)

print("\n🔍 GPU环境检查:")
print("="*60)
gpu_checks = env_manager.check_gpu_environment()

print("\n⚙️ .bashrc配置:")
print("="*60)
bashrc_config = env_manager.generate_bashrc_config()
print(bashrc_config)

## 6. 任务提交和监控

### 6.1 后台任务管理

在服务器上运行长时间任务时，需要确保任务不会因为SSH连接断开而中断：

### 6.2 常用任务管理工具

In [ ]:
import re
import signal

class TaskManager:
    """服务器任务管理器"""
    
    def __init__(self):
        self.active_tasks = {}
    
    def generate_task_scripts(self):
        """生成各种任务管理脚本"""
        
        scripts = {}
        
        # 1. Screen任务脚本
        scripts['screen_training'] = """#!/bin/bash
# Screen训练脚本 - train_with_screen.sh

PROJECT_NAME="mnist_training"
SCRIPT_PATH="train.py"

# 创建screen会话并运行训练
screen -dmS $PROJECT_NAME bash -c "
    echo '🚀 开始训练任务...'
    echo '📅 开始时间: $(date)'
    
    # 激活环境
    conda activate ml_research
    
    # 设置GPU
    export CUDA_VISIBLE_DEVICES=0
    
    # 运行训练
    python $SCRIPT_PATH --epochs 100 --batch-size 64 > training.log 2>&1
    
    echo '✅ 训练完成: $(date)'
    echo '按任意键退出...'
    read
"

echo "✅ Screen会话 '$PROJECT_NAME' 已启动"
echo "🔍 查看状态: screen -list"
echo "🔗 连接会话: screen -r $PROJECT_NAME"
echo "📋 分离会话: Ctrl+A, D"
"""

        # 2. tmux任务脚本
        scripts['tmux_training'] = """#!/bin/bash
# tmux训练脚本 - train_with_tmux.sh

SESSION_NAME="ml_training"
SCRIPT_PATH="train.py"

# 创建tmux会话
tmux new-session -d -s $SESSION_NAME

# 在会话中运行命令
tmux send-keys -t $SESSION_NAME "conda activate ml_research" C-m
tmux send-keys -t $SESSION_NAME "export CUDA_VISIBLE_DEVICES=0" C-m
tmux send-keys -t $SESSION_NAME "echo '🚀 开始训练: $(date)'" C-m
tmux send-keys -t $SESSION_NAME "python $SCRIPT_PATH --config config.yaml" C-m

echo "✅ tmux会话 '$SESSION_NAME' 已启动"
echo "🔍 查看会话: tmux ls"
echo "🔗 连接会话: tmux attach -t $SESSION_NAME"
echo "📋 分离会话: Ctrl+B, D"
"""

        # 3. nohup任务脚本
        scripts['nohup_training'] = """#!/bin/bash
# nohup训练脚本 - train_with_nohup.sh

SCRIPT_PATH="train.py"
LOG_FILE="training_$(date +%Y%m%d_%H%M%S).log"

echo "🚀 使用nohup启动训练任务..."

# 激活环境并运行
nohup bash -c "
    source activate ml_research
    export CUDA_VISIBLE_DEVICES=0
    python $SCRIPT_PATH --epochs 100 --batch-size 64
" > $LOG_FILE 2>&1 &

PID=$!
echo "✅ 任务已启动, PID: $PID"
echo "📄 日志文件: $LOG_FILE"
echo "🔍 监控日志: tail -f $LOG_FILE"
echo "⏹️  停止任务: kill $PID"

# 保存PID到文件
echo $PID > training.pid
"""

        # 4. 任务监控脚本
        scripts['monitor_tasks'] = """#!/bin/bash
# 任务监控脚本 - monitor.sh

# 颜色定义
RED='\\033[0;31m'
GREEN='\\033[0;32m'
YELLOW='\\033[1;33m'
BLUE='\\033[0;34m'
NC='\\033[0m'

echo_info() { echo -e "${GREEN}[INFO]${NC} $1"; }
echo_warn() { echo -e "${YELLOW}[WARN]${NC} $1"; }
echo_error() { echo -e "${RED}[ERROR]${NC} $1"; }

# 显示系统资源
show_resources() {
    echo_info "系统资源使用情况:"
    echo "========================="
    
    # CPU和内存
    echo "💻 CPU和内存:"
    top -bn1 | grep "Cpu(s)" | sed "s/.*, *\\([0-9.]*\\)%* id.*/\\1/" | awk '{print "CPU使用率: " 100-$1"%"}'
    free -h | awk '/Mem:/ {print "内存使用: " $3 "/" $2 " (" $3/$2*100 "%)"}'
    
    # GPU
    if command -v nvidia-smi &> /dev/null; then
        echo "🎮 GPU状态:"
        nvidia-smi --query-gpu=index,name,memory.used,memory.total,utilization.gpu --format=csv,noheader,nounits | \\
        awk -F, '{printf "GPU%s: %s, 内存: %sMB/%sMB (%.1f%%), 利用率: %s%%\\n", $1, $2, $3, $4, $3/$4*100, $5}'
    fi
    
    # 磁盘
    echo "💾 磁盘使用:"
    df -h | grep -E '^/dev/' | awk '{print $6 ": " $3 "/" $2 " (" $5 ")"}'
    echo ""
}

# 显示训练进程
show_training_processes() {
    echo_info "Python训练进程:"
    echo "========================="
    
    ps aux | grep python | grep -v grep | while read line; do
        echo "$line"
    done
    echo ""
}

# 显示Screen会话
show_screen_sessions() {
    echo_info "Screen会话:"
    echo "========================="
    screen -list 2>/dev/null || echo "没有活跃的Screen会话"
    echo ""
}

# 显示tmux会话
show_tmux_sessions() {
    echo_info "tmux会话:"
    echo "========================="
    tmux ls 2>/dev/null || echo "没有活跃的tmux会话"
    echo ""
}

# 显示最近的日志
show_recent_logs() {
    echo_info "最近的训练日志 (最后10行):"
    echo "========================="
    
    find . -name "*.log" -mtime -1 -type f | head -5 | while read logfile; do
        echo "📄 $logfile:"
        tail -3 "$logfile" 2>/dev/null | sed 's/^/  /'
        echo ""
    done
}

# 主监控循环
monitor_loop() {
    while true; do
        clear
        echo "🖥️  服务器监控 - $(date)"
        echo "=============================================="
        
        show_resources
        show_training_processes
        show_screen_sessions
        show_tmux_sessions
        show_recent_logs
        
        echo "🔄 每30秒自动刷新... (Ctrl+C 退出)"
        sleep 30
    done
}

# 主程序
case "$1" in
    "resources"|"res")
        show_resources
        ;;
    "processes"|"proc")
        show_training_processes
        ;;
    "sessions"|"sess")
        show_screen_sessions
        show_tmux_sessions
        ;;
    "logs")
        show_recent_logs
        ;;
    "monitor"|"")
        monitor_loop
        ;;
    *)
        echo "用法: $0 {resources|processes|sessions|logs|monitor}"
        echo "  resources  - 显示系统资源"
        echo "  processes  - 显示训练进程"
        echo "  sessions   - 显示screen/tmux会话"
        echo "  logs       - 显示最近日志"
        echo "  monitor    - 连续监控 (默认)"
        exit 1
        ;;
esac
"""

        return scripts
    
    def generate_gpu_management_script(self):
        """生成GPU管理脚本"""
        
        script = """#!/bin/bash
# GPU管理脚本 - gpu_manager.sh

# 颜色定义
RED='\\033[0;31m'
GREEN='\\033[0;32m'
YELLOW='\\033[1;33m'
BLUE='\\033[0;34m'
NC='\\033[0m'

# 显示GPU状态
show_gpu_status() {
    echo -e "${GREEN}🎮 GPU状态总览${NC}"
    echo "================================"
    
    if ! command -v nvidia-smi &> /dev/null; then
        echo -e "${RED}❌ nvidia-smi 命令不可用${NC}"
        return 1
    fi
    
    # 基本GPU信息
    nvidia-smi --query-gpu=index,name,driver_version,memory.total --format=csv
    echo ""
    
    # 详细状态
    echo -e "${BLUE}📊 详细状态:${NC}"
    nvidia-smi
    echo ""
}

# 显示GPU进程
show_gpu_processes() {
    echo -e "${GREEN}🔍 GPU进程详情${NC}"
    echo "================================"
    
    nvidia-smi pmon -i 0 -c 1 2>/dev/null || {
        echo "使用nvidia-smi查看进程:"
        nvidia-smi --query-compute-apps=pid,process_name,gpu_instance_id,used_memory --format=csv
    }
    echo ""
}

# 选择可用GPU
select_gpu() {
    echo -e "${GREEN}🎯 选择GPU设备${NC}"
    echo "================================"
    
    # 显示每个GPU的内存使用
    nvidia-smi --query-gpu=index,name,memory.used,memory.total --format=csv,noheader,nounits | \\
    while IFS=, read -r idx name mem_used mem_total; do
        usage_percent=$(echo "scale=1; $mem_used * 100 / $mem_total" | bc -l 2>/dev/null || echo "0")
        
        if (( $(echo "$usage_percent < 10" | bc -l) )); then
            status="${GREEN}可用${NC}"
        elif (( $(echo "$usage_percent < 50" | bc -l) )); then
            status="${YELLOW}部分占用${NC}"
        else
            status="${RED}繁忙${NC}"
        fi
        
        echo -e "GPU$idx: $name | 内存: ${mem_used}MB/${mem_total}MB (${usage_percent}%) | $status"
    done
    
    echo ""
    echo "💡 设置GPU使用方法:"
    echo "export CUDA_VISIBLE_DEVICES=0    # 使用GPU 0"
    echo "export CUDA_VISIBLE_DEVICES=0,1  # 使用GPU 0和1"
    echo "export CUDA_VISIBLE_DEVICES=\"\"   # 不使用GPU"
}

# 清理GPU进程
cleanup_gpu() {
    echo -e "${YELLOW}⚠️  清理GPU进程${NC}"
    echo "================================"
    
    echo "当前GPU进程:"
    nvidia-smi --query-compute-apps=pid,process_name --format=csv
    
    echo ""
    echo "请谨慎操作! 这将强制终止GPU进程。"
    read -p "确认清理僵死进程? (y/N): " confirm
    
    if [[ $confirm == [yY] ]]; then
        # 获取GPU进程PID并询问是否终止
        nvidia-smi --query-compute-apps=pid --format=csv,noheader | while read pid; do
            if [[ -n "$pid" && "$pid" != "pid" ]]; then
                echo "终止进程 PID: $pid"
                kill -9 $pid 2>/dev/null || echo "无法终止进程 $pid"
            fi
        done
        echo -e "${GREEN}✅ GPU清理完成${NC}"
    else
        echo "操作已取消"
    fi
}

# 监控GPU使用
monitor_gpu() {
    echo -e "${GREEN}📈 GPU实时监控${NC}"
    echo "================================"
    echo "按 Ctrl+C 退出监控"
    echo ""
    
    # 使用watch命令实时监控
    watch -n 2 "nvidia-smi --query-gpu=index,name,temperature.gpu,power.draw,memory.used,memory.total,utilization.gpu --format=csv,noheader | \\
    awk -F, '{printf \"GPU%s: %s\\n  温度: %s°C | 功耗: %s | 内存: %sMB/%sMB | 利用率: %s%%\\n\\n\", \\$1, \\$2, \\$3, \\$4, \\$5, \\$6, \\$7}'"
}

# 主程序
case "$1" in
    "status"|"")
        show_gpu_status
        ;;
    "processes"|"proc")
        show_gpu_processes
        ;;
    "select"|"sel")
        select_gpu
        ;;
    "cleanup"|"clean")
        cleanup_gpu
        ;;
    "monitor"|"mon")
        monitor_gpu
        ;;
    *)
        echo "用法: $0 {status|processes|select|cleanup|monitor}"
        echo "  status     - 显示GPU状态 (默认)"
        echo "  processes  - 显示GPU进程"
        echo "  select     - 选择可用GPU"
        echo "  cleanup    - 清理GPU进程"
        echo "  monitor    - 实时监控GPU"
        exit 1
        ;;
esac
"""
        
        return script

# 创建任务管理器
task_manager = TaskManager()

print("🛠️ 任务管理脚本集合:")
print("="*60)

# 生成任务脚本
task_scripts = task_manager.generate_task_scripts()

for script_name, script_content in task_scripts.items():
    print(f"\n📄 {script_name}.sh:")
    print("-" * 40)
    print(script_content)

print("\n🎮 GPU管理脚本:")
print("="*60)
gpu_script = task_manager.generate_gpu_management_script()
print(gpu_script)

print("\n💡 使用建议:")
print("1. 保存这些脚本到服务器")
print("2. chmod +x *.sh  # 添加执行权限")
print("3. 根据实际项目路径修改脚本")
print("4. 选择合适的任务管理工具 (screen/tmux/nohup)")

## 7. SLURM 作业调度系统

SLURM (Simple Linux Utility for Resource Management) 是很多高性能计算集群使用的作业调度系统。

### 7.1 SLURM 基础概念

- **节点 (Node)**: 集群中的计算机
- **分区 (Partition)**: 节点组，类似队列
- **作业 (Job)**: 提交的计算任务
- **作业步 (Job Step)**: 作业中的并行任务

### 7.2 常用 SLURM 命令

In [ ]:
class SlurmManager:
    """SLURM作业管理器"""
    
    def __init__(self):
        self.common_partitions = ['gpu', 'cpu', 'highmem', 'debug']
        
    def generate_slurm_templates(self):
        """生成SLURM作业脚本模板"""
        
        templates = {}
        
        # 1. 基础GPU训练脚本
        templates['gpu_training'] = """#!/bin/bash
#SBATCH --job-name=mnist_training          # 作业名称
#SBATCH --partition=gpu                    # 分区名称
#SBATCH --nodes=1                          # 节点数
#SBATCH --ntasks-per-node=1                # 每个节点的任务数
#SBATCH --cpus-per-task=4                  # 每个任务的CPU核数
#SBATCH --gres=gpu:1                       # GPU资源 (1块GPU)
#SBATCH --mem=16G                          # 内存需求
#SBATCH --time=24:00:00                    # 最大运行时间 (24小时)
#SBATCH --output=logs/slurm_%j.out         # 标准输出文件
#SBATCH --error=logs/slurm_%j.err          # 错误输出文件
#SBATCH --mail-type=ALL                    # 邮件通知类型
#SBATCH --mail-user=your_email@university.edu

# 打印作业信息
echo "作业开始时间: $(date)"
echo "作业ID: $SLURM_JOB_ID"
echo "节点名称: $SLURM_NODELIST"
echo "GPU设备: $CUDA_VISIBLE_DEVICES"

# 创建日志目录
mkdir -p logs

# 激活环境
source ~/.bashrc
conda activate ml_research

# 设置环境变量
export PYTHONPATH=$HOME/projects:$PYTHONPATH

# 运行训练脚本
echo "开始训练..."
python train.py \\
    --epochs 100 \\
    --batch-size 64 \\
    --learning-rate 0.001 \\
    --output-dir ./outputs \\
    --log-dir ./logs

echo "训练完成时间: $(date)"
echo "作业结束"
"""

        # 2. 多GPU训练脚本
        templates['multi_gpu_training'] = """#!/bin/bash
#SBATCH --job-name=multi_gpu_training
#SBATCH --partition=gpu
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=8
#SBATCH --gres=gpu:4                       # 4块GPU
#SBATCH --mem=32G
#SBATCH --time=48:00:00
#SBATCH --output=logs/multi_gpu_%j.out
#SBATCH --error=logs/multi_gpu_%j.err

echo "多GPU训练作业开始: $(date)"
echo "可用GPU: $CUDA_VISIBLE_DEVICES"

# 激活环境
conda activate ml_research

# 使用PyTorch DistributedDataParallel
python -m torch.distributed.launch \\
    --nproc_per_node=4 \\
    --use_env \\
    train_distributed.py \\
    --epochs 100 \\
    --batch-size 256 \\
    --world-size 4

echo "多GPU训练完成: $(date)"
"""

        # 3. 参数扫描脚本
        templates['hyperparameter_sweep'] = """#!/bin/bash
#SBATCH --job-name=hyperparam_sweep
#SBATCH --partition=gpu
#SBATCH --array=1-20                       # 作业数组 (20个参数组合)
#SBATCH --cpus-per-task=2
#SBATCH --gres=gpu:1
#SBATCH --mem=8G
#SBATCH --time=12:00:00
#SBATCH --output=logs/sweep_%A_%a.out      # %A是数组作业ID, %a是任务ID
#SBATCH --error=logs/sweep_%A_%a.err

# 参数网格
learning_rates=(0.001 0.003 0.01 0.03)
batch_sizes=(32 64 128 256)
optimizers=("adam" "sgd" "adamw")

# 计算当前任务的参数组合
lr_idx=$(( ($SLURM_ARRAY_TASK_ID - 1) % 4 ))
bs_idx=$(( (($SLURM_ARRAY_TASK_ID - 1) / 4) % 4 ))
opt_idx=$(( (($SLURM_ARRAY_TASK_ID - 1) / 16) % 3 ))

LR=${learning_rates[$lr_idx]}
BS=${batch_sizes[$bs_idx]}
OPT=${optimizers[$opt_idx]}

echo "任务 $SLURM_ARRAY_TASK_ID: LR=$LR, BS=$BS, OPT=$OPT"

# 激活环境
conda activate ml_research

# 运行实验
python train.py \\
    --learning-rate $LR \\
    --batch-size $BS \\
    --optimizer $OPT \\
    --experiment-name "sweep_${SLURM_ARRAY_TASK_ID}" \\
    --output-dir "./outputs/sweep_${SLURM_ARRAY_TASK_ID}"

echo "实验 $SLURM_ARRAY_TASK_ID 完成: $(date)"
"""

        # 4. CPU密集型任务
        templates['cpu_intensive'] = """#!/bin/bash
#SBATCH --job-name=cpu_task
#SBATCH --partition=cpu                    # CPU分区
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=16                 # 16个CPU核
#SBATCH --mem=64G                          # 64GB内存
#SBATCH --time=72:00:00                    # 3天
#SBATCH --output=logs/cpu_%j.out
#SBATCH --error=logs/cpu_%j.err

echo "CPU密集型任务开始: $(date)"
echo "可用CPU核数: $SLURM_CPUS_PER_TASK"

# 激活环境
conda activate ml_research

# 设置OpenMP线程数
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# 运行CPU密集型任务 (如数据预处理)
python preprocess_data.py \\
    --num-workers $SLURM_CPUS_PER_TASK \\
    --input-dir ./raw_data \\
    --output-dir ./processed_data

echo "CPU任务完成: $(date)"
"""

        return templates
    
    def generate_slurm_commands_reference(self):
        """生成SLURM命令参考"""
        
        commands = {
            "作业提交": {
                "sbatch script.sh": "提交作业脚本",
                "sbatch --dependency=afterok:12345 script.sh": "依赖作业12345成功后运行",
                "sbatch --begin=2024-01-01T10:00:00 script.sh": "定时提交作业"
            },
            
            "作业查询": {
                "squeue": "查看所有作业队列",
                "squeue -u $USER": "查看当前用户的作业",
                "squeue -j 12345": "查看特定作业",
                "sinfo": "查看分区和节点信息",
                "sinfo -p gpu": "查看GPU分区信息"
            },
            
            "作业控制": {
                "scancel 12345": "取消作业12345",
                "scancel -u $USER": "取消当前用户所有作业",
                "scontrol hold 12345": "暂停作业",
                "scontrol release 12345": "释放暂停的作业"
            },
            
            "作业信息": {
                "sacct": "查看作业历史",
                "sacct -j 12345": "查看特定作业详情",
                "sstat 12345": "查看运行中作业状态",
                "scontrol show job 12345": "显示作业详细信息"
            },
            
            "资源查询": {
                "sinfo -N": "查看所有节点",
                "sinfo -N -p gpu": "查看GPU节点",
                "squeue -t RUNNING": "查看正在运行的作业",
                "squeue -t PENDING": "查看等待中的作业"
            }
        }
        
        return commands
    
    def generate_slurm_management_script(self):
        """生成SLURM作业管理脚本"""
        
        script = """#!/bin/bash
# SLURM作业管理脚本 - slurm_manager.sh

# 颜色定义
RED='\\033[0;31m'
GREEN='\\033[0;32m'
YELLOW='\\033[1;33m'
BLUE='\\033[0;34m'
NC='\\033[0m'

# 显示我的作业
show_my_jobs() {
    echo -e "${GREEN}📋 我的作业状态${NC}"
    echo "================================"
    
    squeue -u $USER --format="%.8i %.12j %.8u %.10T %.10M %.6D %R" | \\
    awk 'NR==1{print $0; print "================================"} NR>1{print $0}'
    
    echo ""
    echo "作业状态说明:"
    echo "  PD - PENDING (等待中)"
    echo "  R  - RUNNING (运行中)"
    echo "  CG - COMPLETING (完成中)"
    echo "  CD - COMPLETED (已完成)"
    echo "  F  - FAILED (失败)"
    echo ""
}

# 显示分区信息
show_partitions() {
    echo -e "${GREEN}🖥️  分区和节点信息${NC}"
    echo "================================"
    
    sinfo --format="%.15P %.5a %.10l %.6D %.6t %.8C %.8G %.15N" | \\
    awk 'NR==1{print $0; print "================================"} NR>1{print $0}'
    
    echo ""
}

# 显示GPU节点详情
show_gpu_nodes() {
    echo -e "${GREEN}🎮 GPU节点状态${NC}"
    echo "================================"
    
    sinfo -N -p gpu --format="%.15N %.10T %.4c %.8m %.25G %.15P" | \\
    awk 'NR==1{print $0; print "================================"} NR>1{print $0}'
    
    echo ""
}

# 提交训练作业
submit_training() {
    echo -e "${GREEN}🚀 提交训练作业${NC}"
    echo "================================"
    
    # 检查必要文件
    if [ ! -f "train.py" ]; then
        echo -e "${RED}❌ 找不到 train.py 文件${NC}"
        return 1
    fi
    
    # 创建日志目录
    mkdir -p logs
    
    # 生成作业脚本
    cat > submit_job.sh << 'EOF'
#!/bin/bash
#SBATCH --job-name=ml_training
#SBATCH --partition=gpu
#SBATCH --gres=gpu:1
#SBATCH --cpus-per-task=4
#SBATCH --mem=16G
#SBATCH --time=24:00:00
#SBATCH --output=logs/train_%j.out
#SBATCH --error=logs/train_%j.err

echo "作业开始: $(date)"
echo "节点: $SLURM_NODELIST"
echo "GPU: $CUDA_VISIBLE_DEVICES"

conda activate ml_research
python train.py

echo "作业结束: $(date)"
EOF
    
    # 提交作业
    job_id=$(sbatch submit_job.sh | grep -o '[0-9]\\+')
    
    if [ $? -eq 0 ]; then
        echo -e "${GREEN}✅ 作业已提交, ID: $job_id${NC}"
        echo "📄 查看输出: tail -f logs/train_${job_id}.out"
        echo "❌ 查看错误: tail -f logs/train_${job_id}.err"
        echo "⏹️  取消作业: scancel $job_id"
    else
        echo -e "${RED}❌ 作业提交失败${NC}"
    fi
}

# 监控作业
monitor_job() {
    if [ $# -eq 0 ]; then
        echo "用法: $0 monitor <job_id>"
        return 1
    fi
    
    job_id=$1
    echo -e "${GREEN}📊 监控作业 $job_id${NC}"
    echo "================================"
    
    # 检查作业是否存在
    if ! squeue -j $job_id &>/dev/null; then
        echo -e "${RED}❌ 找不到作业 $job_id${NC}"
        return 1
    fi
    
    # 显示作业详情
    scontrol show job $job_id | grep -E "(JobId|JobName|JobState|RunTime|TimeLimit|NodeList)"
    
    echo ""
    echo -e "${BLUE}📄 实时日志 (Ctrl+C 退出):${NC}"
    
    # 查找日志文件
    log_file=$(find logs -name "*${job_id}.out" 2>/dev/null | head -1)
    if [ -n "$log_file" ]; then
        tail -f "$log_file"
    else
        echo "日志文件尚未创建..."
        sleep 5
        log_file=$(find logs -name "*${job_id}.out" 2>/dev/null | head -1)
        [ -n "$log_file" ] && tail -f "$log_file"
    fi
}

# 清理完成的作业
cleanup_jobs() {
    echo -e "${GREEN}🧹 清理已完成的作业${NC}"
    echo "================================"
    
    # 显示已完成的作业
    completed_jobs=$(sacct --format=JobID,JobName,State,ExitCode --state=COMPLETED,FAILED --starttime=now-7days --noheader | wc -l)
    
    echo "最近7天内完成的作业数: $completed_jobs"
    
    # 清理旧的日志文件
    echo "清理7天前的日志文件..."
    find logs -name "*.out" -o -name "*.err" -mtime +7 -delete 2>/dev/null
    
    echo -e "${GREEN}✅ 清理完成${NC}"
}

# 主程序
case "$1" in
    "jobs"|"")
        show_my_jobs
        ;;
    "partitions"|"part")
        show_partitions
        ;;
    "gpu")
        show_gpu_nodes
        ;;
    "submit")
        submit_training
        ;;
    "monitor"|"mon")
        monitor_job $2
        ;;
    "cleanup")
        cleanup_jobs
        ;;
    *)
        echo "用法: $0 {jobs|partitions|gpu|submit|monitor|cleanup}"
        echo "  jobs       - 显示我的作业 (默认)"
        echo "  partitions - 显示分区信息"
        echo "  gpu        - 显示GPU节点"
        echo "  submit     - 提交训练作业"
        echo "  monitor    - 监控作业 <job_id>"
        echo "  cleanup    - 清理旧作业和日志"
        exit 1
        ;;
esac
"""
        
        return script

# 创建SLURM管理器
slurm_manager = SlurmManager()

print("🎯 SLURM作业脚本模板:")
print("="*60)

# 生成模板
templates = slurm_manager.generate_slurm_templates()
for name, template in templates.items():
    print(f"\n📄 {name}.slurm:")
    print("-" * 40)
    print(template)

print("\n📚 SLURM命令参考:")
print("="*60)

# 生成命令参考
commands = slurm_manager.generate_slurm_commands_reference()
for category, cmd_dict in commands.items():
    print(f"\n📋 {category}:")
    print("-" * 30)
    for cmd, desc in cmd_dict.items():
        print(f"  {cmd:<35} # {desc}")

print("\n🛠️ SLURM管理脚本:")
print("="*60)
management_script = slurm_manager.generate_slurm_management_script()
print(management_script)

## 8. 服务器工作流最佳实践

### 8.1 项目组织结构

在服务器上保持良好的项目组织结构非常重要：

```
~/projects/
├── project1/
│   ├── data/
│   │   ├── raw/          # 原始数据
│   │   └── processed/    # 处理后数据
│   ├── src/              # 源代码
│   ├── configs/          # 配置文件
│   ├── scripts/          # 脚本文件
│   ├── logs/             # 日志文件
│   ├── checkpoints/      # 模型检查点
│   ├── results/          # 结果输出
│   └── environment.yml   # 环境配置
└── shared_utils/         # 共享工具
```

### 8.2 数据管理策略

In [ ]:
class ServerBestPractices:
    """服务器使用最佳实践指南"""
    
    def __init__(self):
        self.practices = {}
        
    def data_management_guide(self):
        """数据管理指南"""
        
        guide = """
# 数据管理最佳实践

## 1. 数据存储策略
```bash
# 创建规范的数据目录结构
mkdir -p ~/data/{raw,processed,cache,backup}

# 使用符号链接共享大型数据集
ln -s /shared/datasets/imagenet ~/data/imagenet

# 定期备份重要数据
rsync -av ~/projects/important_results/ ~/backup/$(date +%Y%m%d)/
```

## 2. 数据传输优化
```bash
# 使用压缩传输大文件
tar -czf - large_dataset/ | ssh server 'cd ~/data && tar -xzf -'

# 断点续传大文件
rsync -avz --partial --progress large_file.tar.gz server:~/data/

# 并行传输多个文件
find . -name "*.txt" | parallel -j4 scp {} server:~/data/
```

## 3. 磁盘空间管理
```bash
# 检查磁盘使用
df -h
du -sh ~/projects/*

# 清理临时文件
find ~/projects -name "*.tmp" -delete
find ~/projects -name "__pycache__" -type d -exec rm -rf {} +

# 压缩旧日志
find ~/projects -name "*.log" -mtime +30 -exec gzip {} \\;
```
"""
        
        return guide
    
    def performance_optimization_guide(self):
        """性能优化指南"""
        
        guide = """
# 性能优化最佳实践

## 1. CPU优化
```python
import os
import torch

# 设置OpenMP线程数
os.environ['OMP_NUM_THREADS'] = '4'

# PyTorch数据加载优化
train_loader = DataLoader(
    dataset,
    batch_size=64,
    num_workers=4,        # 根据CPU核心数调整
    pin_memory=True,      # GPU训练时启用
    persistent_workers=True  # 保持worker进程
)
```

## 2. 内存优化
```python
# 梯度检查点 (节省内存)
import torch.utils.checkpoint as checkpoint

class Model(nn.Module):
    def forward(self, x):
        x = checkpoint.checkpoint(self.heavy_layer, x)
        return x

# 清理GPU缓存
torch.cuda.empty_cache()

# 混合精度训练
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()
with autocast():
    output = model(input)
    loss = criterion(output, target)
```

## 3. 网络I/O优化
```bash
# 启用SSH连接复用
echo "Host *
    ControlMaster auto
    ControlPath ~/.ssh/sockets/%r@%h-%p
    ControlPersist 600" >> ~/.ssh/config

mkdir -p ~/.ssh/sockets

# 使用rsync增量同步
rsync -avz --delete --exclude='*.pyc' local/ server:remote/
```
"""
        
        return guide
    
    def security_best_practices(self):
        """安全最佳实践"""
        
        guide = """
# 安全最佳实践

## 1. SSH安全配置
```bash
# 生成强密钥
ssh-keygen -t ed25519 -a 100 -f ~/.ssh/server_key

# 配置SSH客户端
cat >> ~/.ssh/config << EOF
Host secure-server
    HostName server.university.edu
    User your_username
    IdentityFile ~/.ssh/server_key
    PasswordAuthentication no
    PubkeyAuthentication yes
    ServerAliveInterval 60
EOF

# 设置正确的权限
chmod 700 ~/.ssh
chmod 600 ~/.ssh/*
chmod 644 ~/.ssh/*.pub
```

## 2. 敏感信息管理
```bash
# 使用环境变量存储敏感信息
echo "export API_KEY='your_secret_key'" >> ~/.bashrc_private
echo "source ~/.bashrc_private" >> ~/.bashrc

# 在代码中使用
import os
api_key = os.environ.get('API_KEY')
```

## 3. 文件权限管理
```bash
# 设置默认权限
umask 022

# 保护敏感文件
chmod 600 config_with_secrets.yaml
chmod 700 private_scripts/

# 共享项目目录
chmod 755 ~/projects/shared_project/
```
"""
        
        return guide
    
    def troubleshooting_guide(self):
        """故障排除指南"""
        
        guide = """
# 故障排除指南

## 1. 连接问题
### SSH连接超时
```bash
# 检查网络连通性
ping server.university.edu

# 使用详细模式诊断
ssh -v username@server

# 检查SSH服务状态
ssh username@server 'sudo systemctl status sshd'
```

### 连接频繁断开
```bash
# 客户端配置保持连接
echo "ServerAliveInterval 60
ServerAliveCountMax 3" >> ~/.ssh/config

# 服务器端配置 (需要管理员权限)
# 在 /etc/ssh/sshd_config 中添加:
# ClientAliveInterval 60
# ClientAliveCountMax 3
```

## 2. 环境问题
### Conda环境问题
```bash
# 重新初始化Conda
conda init bash
source ~/.bashrc

# 清理损坏的环境
conda remove --name broken_env --all
conda clean --all

# 重建环境
conda env create -f environment.yml
```

### Python包冲突
```bash
# 检查包依赖
pip check

# 创建新的清洁环境
conda create -n fresh_env python=3.9
conda activate fresh_env
pip install -r requirements.txt
```

## 3. 资源问题
### GPU内存不足
```python
# 检查GPU内存使用
import torch
print(f"GPU内存: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")
print(f"已用内存: {torch.cuda.memory_allocated(0)/1e9:.1f}GB")

# 清理GPU缓存
torch.cuda.empty_cache()

# 减少批次大小
batch_size = 32  # 从64减少到32
```

### 磁盘空间不足
```bash
# 查找大文件
find ~ -size +1G -type f -exec ls -lh {} \\; | sort -k5 -hr

# 清理缓存
rm -rf ~/.cache/pip
conda clean --all

# 压缩日志文件
find ~/projects -name "*.log" -exec gzip {} \\;
```

## 4. 性能问题
### 训练速度慢
```python
# 检查数据加载瓶颈
import time

start_time = time.time()
for batch in train_loader:
    data_time = time.time() - start_time
    print(f"数据加载时间: {data_time:.3f}s")
    break

# 增加数据加载workers
train_loader = DataLoader(..., num_workers=8)

# 使用混合精度
from torch.cuda.amp import autocast
with autocast():
    output = model(input)
```

### 内存泄漏
```python
# 监控内存使用
import tracemalloc
tracemalloc.start()

# 训练代码...

current, peak = tracemalloc.get_traced_memory()
print(f"当前内存: {current/1e6:.1f}MB, 峰值: {peak/1e6:.1f}MB")
```
"""
        
        return guide
    
    def monitoring_scripts(self):
        """生成监控脚本"""
        
        script = """#!/bin/bash
# 系统监控脚本 - system_monitor.sh

# 创建监控报告
create_report() {
    local report_file="monitor_report_$(date +%Y%m%d_%H%M%S).txt"
    
    echo "=== 系统监控报告 ===" > $report_file
    echo "生成时间: $(date)" >> $report_file
    echo "" >> $report_file
    
    echo "=== 系统信息 ===" >> $report_file
    uname -a >> $report_file
    echo "" >> $report_file
    
    echo "=== CPU使用率 ===" >> $report_file
    top -bn1 | head -20 >> $report_file
    echo "" >> $report_file
    
    echo "=== 内存使用 ===" >> $report_file
    free -h >> $report_file
    echo "" >> $report_file
    
    echo "=== 磁盘使用 ===" >> $report_file
    df -h >> $report_file
    echo "" >> $report_file
    
    if command -v nvidia-smi &> /dev/null; then
        echo "=== GPU状态 ===" >> $report_file
        nvidia-smi >> $report_file
        echo "" >> $report_file
    fi
    
    echo "=== 网络连接 ===" >> $report_file
    ss -tuln >> $report_file
    echo "" >> $report_file
    
    echo "=== 最近的系统日志 ===" >> $report_file
    journalctl --since "1 hour ago" --no-pager | tail -50 >> $report_file
    
    echo "监控报告已保存到: $report_file"
}

# 实时监控
real_time_monitor() {
    echo "开始实时监控 (Ctrl+C 退出)..."
    
    while true; do
        clear
        echo "=== 实时系统监控 - $(date) ==="
        echo ""
        
        echo "CPU和内存:"
        top -bn1 | head -5 | tail -2
        echo ""
        
        echo "磁盘使用:"
        df -h | grep -E '^/dev/'
        echo ""
        
        if command -v nvidia-smi &> /dev/null; then
            echo "GPU状态:"
            nvidia-smi --query-gpu=index,name,memory.used,memory.total,utilization.gpu --format=csv,noheader,nounits
            echo ""
        fi
        
        echo "网络流量:"
        cat /proc/net/dev | grep -E '(eth|wlan)' | head -2
        echo ""
        
        sleep 5
    done
}

# 检查服务器健康状况
health_check() {
    echo "=== 服务器健康检查 ==="
    
    # CPU温度检查
    if command -v sensors &> /dev/null; then
        echo "CPU温度:"
        sensors | grep -E 'Core|temp'
        echo ""
    fi
    
    # 负载检查
    load=$(uptime | awk -F'load average:' '{print $2}' | cut -d, -f1 | xargs)
    cores=$(nproc)
    echo "系统负载: $load / $cores 核"
    
    if (( $(echo "$load > $cores" | bc -l) )); then
        echo "⚠️  警告: 系统负载过高!"
    else
        echo "✅ 系统负载正常"
    fi
    echo ""
    
    # 内存检查
    mem_usage=$(free | awk '/Mem:/ {printf "%.1f", $3/$2 * 100.0}')
    echo "内存使用率: ${mem_usage}%"
    
    if (( $(echo "$mem_usage > 90" | bc -l) )); then
        echo "⚠️  警告: 内存使用率过高!"
    else
        echo "✅ 内存使用正常"
    fi
    echo ""
    
    # 磁盘检查
    echo "磁盘使用率:"
    df -h | awk 'NR>1 && $5+0 > 90 {print "⚠️  " $0 " - 磁盘空间不足!"}'
    df -h | awk 'NR>1 && $5+0 <= 90 {print "✅ " $0}'
    echo ""
    
    # GPU检查
    if command -v nvidia-smi &> /dev/null; then
        echo "GPU状态检查:"
        nvidia-smi --query-gpu=temperature.gpu,power.draw --format=csv,noheader,nounits | \\
        awk -F, '{temp=$1; power=$2; if(temp>80) print "⚠️  GPU温度过高: " temp "°C"; else print "✅ GPU温度正常: " temp "°C"}'
        echo ""
    fi
}

# 主程序
case "$1" in
    "report")
        create_report
        ;;
    "monitor")
        real_time_monitor
        ;;
    "health")
        health_check
        ;;
    *)
        echo "用法: $0 {report|monitor|health}"
        echo "  report  - 生成监控报告"
        echo "  monitor - 实时监控"
        echo "  health  - 健康检查"
        exit 1
        ;;
esac
"""
        
        return script

# 创建最佳实践指南
practices = ServerBestPractices()

print("📊 数据管理指南:")
print("="*60)
print(practices.data_management_guide())

print("\n⚡ 性能优化指南:")
print("="*60)
print(practices.performance_optimization_guide())

print("\n🔒 安全最佳实践:")
print("="*60)
print(practices.security_best_practices())

print("\n🔧 故障排除指南:")
print("="*60)
print(practices.troubleshooting_guide())

print("\n📈 系统监控脚本:")
print("="*60)
print(practices.monitoring_scripts())

## 9. 完整工作流示例

### 9.1 典型的机器学习项目工作流

以下是一个从项目开始到结束的完整服务器工作流示例：

#### 阶段1：项目初始化
1. 在本地开发和测试代码
2. 准备项目结构和配置文件
3. 创建requirements.txt和environment.yml

#### 阶段2：服务器环境搭建
1. 连接服务器并配置SSH
2. 创建项目目录结构
3. 设置Conda环境
4. 上传初始代码

#### 阶段3：数据准备
1. 上传或链接数据集
2. 运行数据预处理脚本
3. 验证数据完整性

#### 阶段4：模型训练
1. 提交训练任务（SLURM或后台运行）
2. 监控训练进度
3. 调整超参数

#### 阶段5：结果分析
1. 下载训练结果
2. 分析模型性能
3. 生成报告和可视化

### 9.2 工作流自动化脚本

In [ ]:
class CompleteWorkflow:
    """完整的服务器工作流管理"""
    
    def __init__(self, project_name, server_config):
        self.project_name = project_name
        self.server_config = server_config
        
    def generate_workflow_script(self):
        """生成完整工作流脚本"""
        
        script = f"""#!/bin/bash
# 完整机器学习项目工作流 - ml_workflow.sh

PROJECT_NAME="{self.project_name}"
SERVER_USER="{self.server_config.get('user', 'username')}"
SERVER_HOST="{self.server_config.get('host', 'server')}"
REMOTE_BASE="/home/$SERVER_USER/projects"
LOCAL_PROJECT="./$PROJECT_NAME"

# 颜色定义
RED='\\033[0;31m'
GREEN='\\033[0;32m'
YELLOW='\\033[1;33m'
BLUE='\\033[0;34m'
NC='\\033[0m'

log_info() {{ echo -e "${{GREEN}}[INFO]${{NC}} $1"; }}
log_warn() {{ echo -e "${{YELLOW}}[WARN]${{NC}} $1"; }}
log_error() {{ echo -e "${{RED}}[ERROR]${{NC}} $1"; }}

# 阶段1: 项目初始化
init_project() {{
    log_info "🚀 初始化项目: $PROJECT_NAME"
    
    # 创建本地项目结构
    mkdir -p $LOCAL_PROJECT/{{data,src,configs,scripts,logs,results}}
    
    # 创建配置文件
    cat > $LOCAL_PROJECT/requirements.txt << 'EOF'
torch>=1.9.0
torchvision
numpy
pandas
matplotlib
scikit-learn
tensorboard
tqdm
pyyaml
EOF

    cat > $LOCAL_PROJECT/environment.yml << 'EOF'
name: {self.project_name}
channels:
  - pytorch
  - conda-forge
  - defaults
dependencies:
  - python=3.9
  - pytorch
  - torchvision
  - numpy
  - pandas
  - matplotlib
  - scikit-learn
  - jupyter
  - pip
  - pip:
    - tensorboard
    - tqdm
    - pyyaml
EOF

    # 创建README
    cat > $LOCAL_PROJECT/README.md << 'EOF'
# {self.project_name}

## 项目描述
机器学习项目模板

## 环境设置
```bash
conda env create -f environment.yml
conda activate {self.project_name}
```

## 使用方法
1. 数据预处理: `python src/preprocess.py`
2. 模型训练: `python src/train.py`
3. 结果评估: `python src/evaluate.py`
EOF

    log_info "✅ 项目结构创建完成"
}}

# 阶段2: 服务器环境搭建
setup_server() {{
    log_info "🖥️  设置服务器环境"
    
    # 创建远程目录
    ssh $SERVER_USER@$SERVER_HOST "mkdir -p $REMOTE_BASE/$PROJECT_NAME/{{data,src,configs,scripts,logs,results,checkpoints}}"
    
    # 上传项目文件
    rsync -avz --progress \\
        --exclude='*.pyc' \\
        --exclude='__pycache__/' \\
        --exclude='.git/' \\
        --exclude='data/raw/' \\
        $LOCAL_PROJECT/ \\
        $SERVER_USER@$SERVER_HOST:$REMOTE_BASE/$PROJECT_NAME/
    
    # 在服务器上创建环境
    ssh $SERVER_USER@$SERVER_HOST "cd $REMOTE_BASE/$PROJECT_NAME && \\
        conda env create -f environment.yml && \\
        echo 'conda activate $PROJECT_NAME' >> ~/.bashrc"
    
    log_info "✅ 服务器环境设置完成"
}}

# 阶段3: 数据准备
prepare_data() {{
    log_info "📊 准备数据"
    
    # 上传数据（如果有的话）
    if [ -d "$LOCAL_PROJECT/data/raw" ]; then
        log_info "上传原始数据..."
        rsync -avz --progress \\
            $LOCAL_PROJECT/data/raw/ \\
            $SERVER_USER@$SERVER_HOST:$REMOTE_BASE/$PROJECT_NAME/data/raw/
    fi
    
    # 在服务器上运行数据预处理
    ssh $SERVER_USER@$SERVER_HOST "cd $REMOTE_BASE/$PROJECT_NAME && \\
        conda activate $PROJECT_NAME && \\
        python -c 'print(\\\"数据预处理开始\\\")' && \\
        echo '数据预处理完成'"
    
    log_info "✅ 数据准备完成"
}}

# 阶段4: 提交训练任务
submit_training() {{
    log_info "🤖 提交训练任务"
    
    # 创建训练脚本
    cat > train_job.sh << 'EOF'
#!/bin/bash
#SBATCH --job-name={self.project_name}_train
#SBATCH --partition=gpu
#SBATCH --gres=gpu:1
#SBATCH --cpus-per-task=4
#SBATCH --mem=16G
#SBATCH --time=24:00:00
#SBATCH --output=logs/train_%j.out
#SBATCH --error=logs/train_%j.err

echo "训练开始: $(date)"
conda activate {self.project_name}
python src/train.py --config configs/default.yaml
echo "训练结束: $(date)"
EOF

    # 上传并提交作业
    scp train_job.sh $SERVER_USER@$SERVER_HOST:$REMOTE_BASE/$PROJECT_NAME/
    
    job_id=$(ssh $SERVER_USER@$SERVER_HOST "cd $REMOTE_BASE/$PROJECT_NAME && sbatch train_job.sh" | grep -o '[0-9]\\+')
    
    if [ -n "$job_id" ]; then
        log_info "✅ 训练任务已提交, 作业ID: $job_id"
        echo $job_id > .job_id
    else
        log_error "❌ 训练任务提交失败"
        return 1
    fi
}}

# 阶段5: 监控训练
monitor_training() {{
    if [ ! -f ".job_id" ]; then
        log_error "❌ 没有找到作业ID文件"
        return 1
    fi
    
    job_id=$(cat .job_id)
    log_info "📊 监控训练任务: $job_id"
    
    # 检查作业状态
    ssh $SERVER_USER@$SERVER_HOST "squeue -j $job_id"
    
    # 实时查看日志
    log_info "📄 实时日志 (Ctrl+C 退出):"
    ssh $SERVER_USER@$SERVER_HOST "tail -f $REMOTE_BASE/$PROJECT_NAME/logs/train_${{job_id}}.out"
}}

# 阶段6: 下载结果
download_results() {{
    log_info "📥 下载训练结果"
    
    # 创建本地结果目录
    mkdir -p $LOCAL_PROJECT/results/$(date +%Y%m%d_%H%M%S)
    
    # 下载结果文件
    rsync -avz --progress \\
        $SERVER_USER@$SERVER_HOST:$REMOTE_BASE/$PROJECT_NAME/checkpoints/ \\
        $LOCAL_PROJECT/results/checkpoints/
    
    rsync -avz --progress \\
        $SERVER_USER@$SERVER_HOST:$REMOTE_BASE/$PROJECT_NAME/logs/ \\
        $LOCAL_PROJECT/results/logs/
    
    # 下载可视化结果
    rsync -avz --progress \\
        $SERVER_USER@$SERVER_HOST:$REMOTE_BASE/$PROJECT_NAME/results/ \\
        $LOCAL_PROJECT/results/
    
    log_info "✅ 结果下载完成"
}}

# 清理函数
cleanup() {{
    log_info "🧹 清理服务器临时文件"
    
    ssh $SERVER_USER@$SERVER_HOST "cd $REMOTE_BASE/$PROJECT_NAME && \\
        find . -name '*.tmp' -delete && \\
        find . -name '__pycache__' -type d -exec rm -rf {{}} + 2>/dev/null || true"
    
    log_info "✅ 清理完成"
}}

# 完整工作流
run_complete_workflow() {{
    log_info "🎯 开始完整工作流"
    
    init_project
    setup_server
    prepare_data
    submit_training
    
    log_info "🎉 工作流启动完成！"
    log_info "📋 后续操作:"
    log_info "  - 监控训练: $0 monitor"
    log_info "  - 下载结果: $0 download"
    log_info "  - 清理文件: $0 cleanup"
}}

# 主程序
case "$1" in
    "init")
        init_project
        ;;
    "setup")
        setup_server
        ;;
    "data")
        prepare_data
        ;;
    "train")
        submit_training
        ;;
    "monitor")
        monitor_training
        ;;
    "download")
        download_results
        ;;
    "cleanup")
        cleanup
        ;;
    "all"|"")
        run_complete_workflow
        ;;
    *)
        echo "用法: $0 {{init|setup|data|train|monitor|download|cleanup|all}}"
        echo "  init     - 初始化项目"
        echo "  setup    - 设置服务器环境"
        echo "  data     - 准备数据"
        echo "  train    - 提交训练任务"
        echo "  monitor  - 监控训练"
        echo "  download - 下载结果"
        echo "  cleanup  - 清理临时文件"
        echo "  all      - 运行完整工作流 (默认)"
        exit 1
        ;;
esac
"""
        
        return script
    
    def generate_project_template(self):
        """生成项目模板文件"""
        
        templates = {}
        
        # 训练脚本模板
        templates['train.py'] = """#!/usr/bin/env python3
\"\"\"
机器学习训练脚本模板
\"\"\"

import os
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import argparse
from pathlib import Path

def load_config(config_path):
    \"\"\"加载配置文件\"\"\"
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

def setup_logging(log_dir):
    \"\"\"设置日志\"\"\"
    log_dir = Path(log_dir)
    log_dir.mkdir(parents=True, exist_ok=True)
    
    writer = SummaryWriter(log_dir)
    return writer

def train_model(config):
    \"\"\"训练模型\"\"\"
    print(f"🚀 开始训练: {{config['model']['name']}}")
    
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用设备: {{device}}")
    
    # 这里添加你的训练代码
    # model = YourModel()
    # optimizer = optim.Adam(model.parameters(), lr=config['training']['lr'])
    # criterion = nn.CrossEntropyLoss()
    
    # 训练循环
    for epoch in range(config['training']['epochs']):
        print(f"Epoch {{epoch+1}}/{{config['training']['epochs']}}")
        
        # 训练代码...
        
        # 保存检查点
        if (epoch + 1) % config['training']['save_frequency'] == 0:
            checkpoint_path = Path(config['paths']['checkpoints']) / f"model_epoch_{{epoch+1}}.pth"
            # torch.save(model.state_dict(), checkpoint_path)
            print(f"✅ 检查点已保存: {{checkpoint_path}}")
    
    print("🎉 训练完成!")

def main():
    parser = argparse.ArgumentParser(description='训练脚本')
    parser.add_argument('--config', type=str, default='configs/default.yaml',
                       help='配置文件路径')
    args = parser.parse_args()
    
    # 加载配置
    config = load_config(args.config)
    
    # 设置日志
    writer = setup_logging(config['paths']['logs'])
    
    # 开始训练
    train_model(config)
    
    writer.close()

if __name__ == '__main__':
    main()
"""

        # 配置文件模板
        templates['default.yaml'] = f"""
# 默认配置文件
project:
  name: "{self.project_name}"
  description: "机器学习项目"

model:
  name: "SimpleCNN"
  num_classes: 10
  dropout: 0.5

training:
  epochs: 100
  batch_size: 64
  learning_rate: 0.001
  optimizer: "adam"
  scheduler: "step"
  save_frequency: 10

data:
  dataset: "MNIST"
  train_split: 0.8
  val_split: 0.1
  test_split: 0.1

paths:
  data: "./data"
  checkpoints: "./checkpoints"
  logs: "./logs"
  results: "./results"

hardware:
  device: "auto"  # auto, cpu, cuda
  num_workers: 4
  pin_memory: true
"""
        
        return templates

# 创建完整工作流管理器
server_config = {
    'user': 'your_username',
    'host': 'gpu-server',
    'port': 22
}

workflow = CompleteWorkflow("my_ml_project", server_config)

print("🔄 完整工作流脚本:")
print("="*60)
workflow_script = workflow.generate_workflow_script()
print(workflow_script)

print("\n📋 项目模板文件:")
print("="*60)
templates = workflow.generate_project_template()

for filename, content in templates.items():
    print(f"\n📄 {filename}:")
    print("-" * 40)
    print(content)

print("\n🎯 使用指南:")
print("="*60)
print("""
1. 保存工作流脚本为 ml_workflow.sh
2. 修改脚本中的服务器配置
3. chmod +x ml_workflow.sh
4. 运行完整工作流: ./ml_workflow.sh all

各阶段可以单独运行:
- 初始化项目: ./ml_workflow.sh init
- 设置服务器: ./ml_workflow.sh setup  
- 准备数据: ./ml_workflow.sh data
- 提交训练: ./ml_workflow.sh train
- 监控训练: ./ml_workflow.sh monitor
- 下载结果: ./ml_workflow.sh download
- 清理文件: ./ml_workflow.sh cleanup
""")

## 10. 总结与进阶学习

### 10.1 核心要点回顾

通过本教程，我们学习了服务器使用的全方位技能：

✅ **SSH连接管理**：配置文件、密钥认证、连接保持
✅ **文件传输优化**：scp、rsync、增量同步、排除模式
✅ **环境配置**：Conda环境管理、包依赖、GPU配置
✅ **任务管理**：screen、tmux、nohup、后台运行
✅ **SLURM系统**：作业提交、资源申请、队列管理
✅ **监控和调试**：资源监控、日志管理、故障排除
✅ **完整工作流**：项目自动化、脚本管理、最佳实践

### 10.2 服务器使用技能对比

| 技能等级 | 基础技能 | 进阶技能 | 专家技能 |
|----------|----------|----------|----------|
| **连接** | SSH基本连接 | 配置文件、密钥管理 | 跳板机、端口转发 |
| **文件** | scp基本传输 | rsync增量同步 | 并行传输、压缩优化 |
| **环境** | 基本包安装 | 虚拟环境管理 | 容器化、环境隔离 |
| **任务** | 命令行运行 | 后台任务管理 | 作业调度、资源优化 |
| **监控** | 基本状态查看 | 实时监控脚本 | 自动化告警、性能分析 |

### 10.3 进阶学习方向

1. **容器技术**：
   - Docker 容器化部署
   - Singularity 集群容器
   - Kubernetes 编排

2. **自动化工具**：
   - Ansible 配置管理
   - Jenkins CI/CD
   - GitHub Actions

3. **监控系统**：
   - Prometheus + Grafana
   - ELK Stack (日志分析)
   - 自定义监控告警

4. **高级网络**：
   - VPN 和隧道技术
   - 负载均衡
   - 分布式存储

### 10.4 推荐资源

- **官方文档**：
  - [SSH 官方手册](https://man.openbsd.org/ssh)
  - [SLURM 文档](https://slurm.schedmd.com/documentation.html)
  - [GNU Screen 用户手册](https://www.gnu.org/software/screen/manual/)

- **实用工具**：
  - [tmux 速查表](https://tmuxcheatsheet.com/)
  - [rsync 详细教程](https://rsync.samba.org/)
  - [htop 系统监控](https://htop.dev/)

- **最佳实践**：
  - [高性能计算最佳实践](https://hpc-wiki.info/)
  - [Linux 服务器管理指南](https://linuxhandbook.com/)
  - [Shell 脚本编程指南](https://tldp.org/LDP/Bash-Beginners-Guide/html/)

恭喜你完成了服务器工作流教程！🎉

现在你已经具备了在服务器上高效进行机器学习研究的技能。记住：
- 🔧 **自动化一切**：能脚本化的就不要手动操作
- 📊 **监控为王**：时刻了解系统和任务状态  
- 🔒 **安全第一**：保护好你的服务器访问凭证
- 📝 **文档记录**：记录你的配置和工作流程
- 🤝 **团队协作**：与团队成员分享最佳实践

祝你在服务器上的机器学习研究取得成功！🚀